In [2]:
import torch
import torchvision as tv
import os
from transformer_flow import Model
import utils
import pathlib
utils.set_random_seed(100)
notebook_output_path = pathlib.Path('runs/notebook')

In [ ]:
dataset = 'mnist'
num_classes = 10
img_size = 28
channel_size = 1

# we use a small model for fast demonstration, increase the model size for better results
patch_size = 4
channels = 128
blocks = 4
layers_per_block = 4
# try different noise levels to see its effect
noise_std = 0.1

batch_size = 256
lr = 2e-3
# increase epochs for better results
epochs = 100
sample_freq = 10

if torch.cuda.is_available():
    device = 'cuda' 
elif torch.backends.mps.is_available():
    device = 'mps' # if on mac
else:
    device = 'cpu' # if mps not available
print(f'using device {device}')

fixed_noise = torch.randn(num_classes * 10, (img_size // patch_size)**2, channel_size * patch_size ** 2, device=device)
fixed_y = torch.arange(num_classes, device=device).view(-1, 1).repeat(1, 10).flatten()

transform = tv.transforms.Compose([
    tv.transforms.Resize((img_size, img_size)),
    tv.transforms.ToTensor(),
    tv.transforms.Normalize((0.5,), (0.5,))
])
data = tv.datasets.MNIST('.', transform=transform, train=True, download=True)
data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=True)

model = Model(in_channels=channel_size, img_size=img_size, patch_size=patch_size, 
              channels=channels, num_blocks=blocks, layers_per_block=layers_per_block,
              num_classes=num_classes).to(device)

optimizer = torch.optim.AdamW(model.parameters(), betas=(0.9, 0.95), lr=lr, weight_decay=1e-4)
lr_schedule = utils.CosineLRSchedule(optimizer, len(data_loader), epochs * len(data_loader), 1e-6, lr)

model_name = f'{patch_size}_{channels}_{blocks}_{layers_per_block}_{noise_std:.2f}'
sample_dir = notebook_output_path / f'{dataset}_samples_{model_name}'
ckpt_file = notebook_output_path / f'{dataset}_model_{model_name}.pth'
sample_dir.mkdir(exist_ok=True, parents=True)

for epoch in range(epochs):
    losses = 0
    for x, y in data_loader:
        x = x.to(device)
        eps = noise_std * torch.randn_like(x)
        x = x + eps
        y = y.to(device)
        optimizer.zero_grad()
        z, outputs, logdets = model(x, y)
        loss = model.get_loss(z, logdets)
        loss.backward()
        optimizer.step()
        lr_schedule.step()
        losses += loss.item()

    print(f"epoch {epoch} lr {optimizer.param_groups[0]['lr']:.6f} loss {losses / len(data_loader):.4f}")
    print('layer norms', ' '.join([f'{z.pow(2).mean():.4f}' for z in outputs]))
    if (epoch + 1) % sample_freq == 0:
        with torch.no_grad():
            samples = model.reverse(fixed_noise, fixed_y)
        tv.utils.save_image(samples, sample_dir / f'samples_{epoch:03d}.png', normalize=True, nrow=10)
        tv.utils.save_image(model.unpatchify(z[:100]), sample_dir / f'latent_{epoch:03d}.png', normalize=True, nrow=10)
        print('sampling complete')
torch.save(model.state_dict(), ckpt_file)

using device cuda
epoch 0 lr 0.002000 loss -0.9851
layer norms 2.5110 2.3136 1.4391 1.0744
epoch 1 lr 0.001999 loss -1.3945
layer norms 1.6760 1.9067 1.3868 1.0188
epoch 2 lr 0.001998 loss -1.4875
layer norms 1.4873 1.6582 1.1648 1.0448
epoch 3 lr 0.001995 loss -1.5321
layer norms 1.4126 1.5511 1.1112 1.0344
epoch 4 lr 0.001992 loss -1.5517
layer norms 1.4126 1.5986 1.2093 1.0449
epoch 5 lr 0.001987 loss -1.5631
layer norms 1.4329 1.6415 1.2954 1.0164
epoch 6 lr 0.001982 loss -1.5656
layer norms 1.5021 1.7343 1.3811 1.0056
epoch 7 lr 0.001975 loss -1.5757
layer norms 1.4603 1.7595 1.4777 1.0003
epoch 8 lr 0.001968 loss -1.5760
layer norms 1.5301 1.8832 1.5883 0.9890
epoch 9 lr 0.001960 loss -1.5792
layer norms 1.5734 1.9935 1.7199 1.0091
sampling complete
epoch 10 lr 0.001950 loss -1.5859
layer norms 1.5777 2.0428 1.7960 0.9815
epoch 11 lr 0.001940 loss -1.5871
layer norms 1.6115 2.1037 1.8727 1.0129
epoch 12 lr 0.001928 loss -1.5906
layer norms 1.6130 2.1483 1.9623 1.0159
epoch 13 lr 

In [ ]:
# now we can also evaluate the model by turning it into a classifier with Bayes rule, p(y|x) = p(y)p(x|y)/p(x)
data = tv.datasets.MNIST('.', transform=transform, train=False, download=False)
data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=False)
num_correct = 0
num_examples = 0
for x, y in data_loader:
    x = x.to(device)
    y = y.to(device)
    eps = noise_std * torch.randn_like(x)
    x = x.repeat(num_classes, 1, 1, 1)
    y_ = torch.arange(num_classes, device=device).view(-1, 1).repeat(1, y.size(0)).flatten()
    with torch.no_grad():
        z, outputs, logdets = model(x, y_)
        losses = 0.5 * z.pow(2).mean(dim=[1, 2]) - logdets # keep the batch dimension
        pred = losses.reshape(num_classes, y.size(0)).argmin(dim=0)
    num_correct += (pred == y).sum()
    num_examples += y.size(0)
print(f'Accuracy %{100 * num_correct / num_examples:.2f}')